In [1]:
import numpy as np
import math
import re
import time

In [2]:
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_datasets as tfds

## Load data

In [3]:
with open("./fr-en/europarl-v7.fr-en.en",mode='r',encoding='utf-8') as f:
    europarl_en = f.read()
    
with open("./fr-en/europarl-v7.fr-en.fr",mode='r',encoding='utf-8') as f:
    europarl_fr = f.read()
    
with open("./fr-en/nonbreaking_prefix.en",mode='r',encoding='utf-8') as f:
    non_breaking_prefix_en = f.read()

with open("./fr-en/nonbreaking_prefix.fr",mode='r',encoding='utf-8') as f:
    non_breaking_prefix_fr = f.read()

In [4]:
europarl_en[:50]

'Resumption of the session\nI declare resumed the se'

## Cleaning

In [5]:
non_breaking_prefix_en = non_breaking_prefix_en.split('\n')
non_breaking_prefix_en = [' ' + pref + '.' for pref in non_breaking_prefix_en]

non_breaking_prefix_fr = non_breaking_prefix_fr.split('\n')
non_breaking_prefix_fr = [' ' + pref + '.' for pref in non_breaking_prefix_fr]

We will need each word and other symbol that we want to keep to be in lower case and separated by spaces, so we can "tokenize" them.

In [ ]:
corpus_en = europarl_en
for prefix in non_breaking_prefix_en:
    corpus_en = corpus_en.replace(prefix, prefix + "###")
corpus_en = re.sub(r"\.(?=[0-9]|[a-z]|[A-Z])", ".###", corpus_en)
corpus_en = re.sub(r"\.###",'',corpus_en)
corpus_en = re.sub(r" +", ' ', corpus_en)
corpus_en = corpus_en.split('\n')

corpus_fr = europarl_fr
for prefix in non_breaking_prefix_fr:
    corpus_fr = corpus_fr.replace(prefix, prefix + "###")
corpus_fr = re.sub(r"\.(?=[0-9]|[a-z]|[A-Z])", ".###", corpus_fr)
corpus_fr = re.sub(r"\.###",'',corpus_fr)
corpus_fr = re.sub(r" +", ' ', corpus_fr)
corpus_fr = corpus_fr.split('\n')

## Tokenizing text

In [ ]:
tokenizer_en = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    corpus_en, target_vocab_size=2**13)
tokenizer_fr = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    corpus_fr, target_vocab_size=2**13)

In [ ]:
VOCAB_SIZE_EN = tokenizer_en.vocab_size + 2
VOCAB_SIZE_FR = tokenizer_fr.vocab_size + 2

In [ ]:
inputs = [[VOCAB_SIZE_EN-2] + tokenizer_en.encode(sentence) + [VOCAB_SIZE_EN-1]
          for sentence in corpus_en]
outputs = [[VOCAB_SIZE_FR-2] + tokenizer_fr.encode(sentence) + [VOCAB_SIZE_FR-1]
           for sentence in corpus_fr]

## Remove too long sentences

In [ ]:
MAX_LENGTH = 20
idx_to_remove = [count for count, sent in enumerate(inputs)
                 if len(sent) > MAX_LENGTH]
for idx in reversed(idx_to_remove):
    del inputs[idx]
    del outputs[idx]
idx_to_remove = [count for count, sent in enumerate(outputs)
                 if len(sent) > MAX_LENGTH]
for idx in reversed(idx_to_remove):
    del inputs[idx]
    del outputs[idx]

## Inputs/outputs creation

As we train with batches, we need each input to have the same length. We pad with the appropriate token, and we will make sure this padding token doesn't interfere with our training later.